In [24]:
import os
import json
import glob
import numpy as np
import pandas as pd
from astroquery.ipac.ned import Ned
from wiserep_api.properties import get_target_property

## Redshift from Host Galaxy

In [21]:
def load_z_dict():
    """Loads the dictionary with the redshift values.
    """
    redshift_file = 'redshifts.json'
    if os.path.isfile(redshift_file) is True:
        with open(redshift_file, 'r') as fp:
            z_dict = json.load(fp)
    else:
        z_dict = {'name':[], 'z':[], 'source':[], 'host_name':[]}
        
    return z_dict

def save_z_dict(z_dict):
    """Saves the dictionary with the redshift values. 
    """
    with open('redshifts.json', 'w') as outfile:
        json.dump(z_dict, outfile)
        
def get_redshift(target, overwrite=False):
    """Obtains the redshift for the given target.
    """
    z_dict = load_z_dict()
    if target in z_dict['name'] and overwrite is False:
        # quit
        return None
    
    host_name = get_target_property(target, 'host')
    if host_name!='':
        try:
            # Get NED redshift
            try:
                z_table = Ned.get_table(host_name, table='redshifts')
            except:
                # try removing white spaces from the name, if there is any
                z_table = Ned.get_table(host_name.replace(' ', ''), 
                                        table='redshifts')
            z = z_table['Published Redshift'].value[0]
            source = 'host'
        except Exception as exc:
            print(f'{target} ({host_name}): {exc}')
            # quit
            return None
    else:
        z = np.nan
        source = ''

    if target not in z_dict['name']:
        z_dict['name'].append(target)
        z_dict['z'].append(z)
        z_dict['source'].append(source)
        z_dict['host_name'].append(host_name)

    elif target in z_dict['name'] and overwrite is True:
        target_id = z_dict['name'].index(target)

        z_dict['z'][target_id] = z
        z_dict['source'][target_id] = source
        z_dict['host_name'][target_id] = host_name
        
    save_z_dict(z_dict)

In [ ]:
get_target_property

In [3]:
peak_spectra = glob.glob('peak_spectra/*')
targets = []
for file in peak_spectra:
    target = os.path.basename(file).split('_')[0]
    if target not in targets:
        targets.append(target)
        
len(targets)

1890

In [22]:
for target in targets:
    get_redshift(target)

2004as (Anon.): The remote service returned the following error message.
ERROR: The object name that you have submitted is not currently recognized by the NED name interpreter.
2006bt (Anon.): The remote service returned the following error message.
ERROR: The object name that you have submitted is not currently recognized by the NED name interpreter.
2003ek (Anon.): The remote service returned the following error message.
ERROR: The object name that you have submitted is not currently recognized by the NED name interpreter.
2008bc (A093831-6358): The remote service returned the following error message.
ERROR: The object name that you have submitted is not currently recognized by the NED name interpreter.
2007qe (Anon.): The remote service returned the following error message.
ERROR: The object name that you have submitted is not currently recognized by the NED name interpreter.
2007ud (A023013-0915): The remote service returned the following error message.
ERROR: The object name that 

In [ ]:
from astroquery.ipac.ned import Ned
Ned.query_object("SDSS J134124.97+3545")

In [23]:
z_dict = load_z_dict()
z_df = pd.DataFrame(z_dict)
z_df.to_csv('redshifts.csv', index=False)
len(z_df)

1735

## Redshift from SNID Templates

In [10]:
directories = glob.glob('spectra/*')
print(f'{len(directories)} SNe')

n_templates = 10
max_phase = +5

8659 SNe


In [36]:
snid_dict = {'name':[], 'snid_z':[], 'snid_z_std':[]}

for directory in directories:
    sn_name = os.path.basename(directory)
    target_files = glob.glob(os.path.join(directory, '*'))
    # SNID outputs
    snid_files = [file for file in target_files if file.endswith('.output')]

    # iterate over all spectra for this SN
    z_list = []
    z_err_list = []
    for templ_file in snid_files:
        df = pd.read_csv(templ_file, nrows=n_templates)
        mean_phase = df.age.mean()

        if np.abs(mean_phase) <= max_phase:
            # this spectrum is at peak
            z_list = z_list + list(df.z.values)
            z_err_list = z_err_list + list(df.zerr.values)
       
    if len(z_list) == 0:
        snid_z = snid_z_std = np.nan
    else:
        # weighted average taking the SNID z and zerr
        w = 1/np.array(z_err_list)**2
        snid_z = np.nansum(w*np.array(z_list))/np.nansum(w)
        #snid_z = np.average(np.array(z_list), 
        #                    weights=1/np.array(z_err_list)**2)
        snid_z = np.nanmean(np.array(z_list))
        snid_z_std = np.nanstd(np.array(z_list))
    
    snid_dict['name'].append(sn_name)
    snid_dict['snid_z'].append(np.round(snid_z, 4))
    snid_dict['snid_z_std'].append(np.round(snid_z_std, 4))
    
snid_z_df = pd.DataFrame(snid_dict)
snid_z_df.dropna(inplace=True)
snid_z_df.to_csv('snid_redshifts.csv', index=False)

/tmp/ipykernel_7532/731335242.py:25: RuntimeWarning: divide by zero encountered in divide
  w = 1/np.array(z_err_list)**2
/tmp/ipykernel_7532/731335242.py:26: RuntimeWarning: invalid value encountered in scalar divide
  snid_z = np.nansum(w*np.array(z_list))/np.nansum(w)
/tmp/ipykernel_7532/731335242.py:25: RuntimeWarning: divide by zero encountered in divide
  w = 1/np.array(z_err_list)**2
/tmp/ipykernel_7532/731335242.py:26: RuntimeWarning: invalid value encountered in scalar divide
  snid_z = np.nansum(w*np.array(z_list))/np.nansum(w)
/tmp/ipykernel_7532/731335242.py:25: RuntimeWarning: divide by zero encountered in divide
  w = 1/np.array(z_err_list)**2
/tmp/ipykernel_7532/731335242.py:26: RuntimeWarning: invalid value encountered in scalar divide
  snid_z = np.nansum(w*np.array(z_list))/np.nansum(w)
/tmp/ipykernel_7532/731335242.py:25: RuntimeWarning: divide by zero encountered in divide
  w = 1/np.array(z_err_list)**2
/tmp/ipykernel_7532/731335242.py:26: RuntimeWarning: invalid v

## Combine Redshifts

In [37]:
master_df = pd.merge(snid_z_df, z_df, how='outer', on='name')
master_df.to_csv('all_redshifts.csv', index=False)
master_df.sort_values(by=['name'], inplace=True, ignore_index=True)
master_df

,name,snid_z,snid_z_std,z,source,host_name
0,1572A,0.0059,0.0051,NaN,NaN,NaN
1,1937D,-0.0027,0.0000,0.002081,host,NGC 1003
2,1960F,0.0222,0.0000,0.005774,host,NGC4496
3,1981B,-0.0028,0.0039,0.006031,host,NGC 4536
4,1986G,0.0274,0.0502,0.001825,host,NGC5128
...,...,...,...,...,...,...
1885,iPTF15xi,0.0439,0.0028,NaN,,
1886,iPTF15zr,0.2046,0.0043,NaN,,
1887,ps15cku,0.0260,0.0037,NaN,NaN,NaN
1888,psnj0252+4656,0.0291,0.0010,0.028086,host,UGC 2351


In [38]:
print(f'There is a total of {len(master_df)} objects:')
print(f'\t {len(master_df[master_df.z.isna() & master_df.snid_z.isna()])} objects do not have redshift at all')
print(f'\t Only {len(master_df.dropna(subset=["z"]))} objects have redshift from host (NED)')

There is a total of 1890 objects:
	 0 objects do not have redshift at all
	 Only 402 objects have redshift from host (NED)
